# Projeto 1 - Ciência dos Dados

Nome: Cesar Ades

Nome: Jonathan Sutton

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'ps4.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com o trabalho!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo ps4.xlsx, tudo certo para prosseguir com o trabalho!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,só queria o meu ps4 😭,0.0
1,eu que comprei no pc e ps4: 🤡🤡🤡 https://t.co/f...,0.0
2,@biceta_rj e eu que só peguei o ps4 em 2019???...,1.0
3,@its_iucas minha herança: desgosto e um ps4,0.0
4,@duduxth1 fall guys eu tenho de graça no ps4,1.0


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@lucas_hunger pq não tem pokémon e zelda pro ps4,0
1,se isso influenciasse no preço do ps4/ps5 eu a...,1
2,quem ai quer me dar um ps4 ?,0
3,@fabricadecobres n sei pq eu n tenho ps4. ou u...,0
4,@dudalbessa falando em tv\neu queria comprar u...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


Nosso produto é o Play Station 4 que é um console para videogames fabricado pela Sony, tem diversos jogos e utiliza controles bluetooth.Consideramos tweets relevantes aqueles que envolviam informações sobre o preço, local de fabricação, performance do produto, jogos que podem ser usados, lançamentos de jogos e falhas que ocorrem com o console e e os produtos relacionados a ele.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
#Definindo as variáveis como categóricas
train['Relevância'] = train['Relevância'].astype('category')
test['Relevância'] = test['Relevância'].astype('category')
train['Relevância'].cat.categories = ['Não relevante','Relevante']  
test['Relevância'].cat.categories = ['Não relevante','Relevante']

In [6]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;/@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed    

Na célula abaixo, fizemos o percentual de tweets relevantes e irrelevantes na parte de treinamentos.

In [7]:
resultados_treino = train.Relevância.value_counts(True)
relevantes_treino = resultados_treino[0]
irrelevantes_treino = resultados_treino[1]
resultados_brutos_treino = train.Relevância.value_counts()
relevantes_brutos_treino = resultados_brutos_treino[0]
irrelevantes_brutos_treino = resultados_brutos_treino[1]
tweets_totais_treino = relevantes_brutos_treino + irrelevantes_brutos_treino

print('Tweets relevantes em Treinamento: {0}'.format(relevantes_brutos_treino))
print('Tweets irrelevantes em Treinamento: {0}'.format(irrelevantes_brutos_treino))
print('Tweets totais em Treinamento: {0}'.format(tweets_totais_treino))
print('Porcentagem de Tweets relevantes em Treinamento: {0:.2f}%'.format(relevantes_treino*100))
print('Porcentagem de Tweets irrelevantes em Treinamento: {0:.2f}%'.format(irrelevantes_treino*100))

Tweets relevantes em Treinamento: 165
Tweets irrelevantes em Treinamento: 133
Tweets totais em Treinamento: 298
Porcentagem de Tweets relevantes em Treinamento: 55.37%
Porcentagem de Tweets irrelevantes em Treinamento: 44.63%


In [8]:
lista_treino = []
for tweet in train.Treinamento:
    lista_treino.append(''.join(cleanup(tweet)))
for tweet in range(len(train.Treinamento)):
    train.Treinamento[tweet] = lista_treino[tweet]

C:\Users\jonat\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Pronto! Agora temos um dataframe sem os sinais de pontuação. Isso ajudará o nosso classificador a fazer a análise! A seguir iremos usar a função split para criar 2 listas com palavras: uma de irrelevantes e outra de relevantes!

In [9]:
relevantes = []         #lista com palavras que aparecem em tweets relevantes
irrelevantes = []       #lista com palavras que aparecem em tweets irrelevantes

#A função extend é parecida com o append, mas põe cada elemento em uma linha. Isso ajuda na hora de montar o dataframe
for tweet in range(len(train)):
    if train.Relevância[tweet] == 'Relevante':
        separador = train.Treinamento[tweet].split()
        relevantes.extend(separador)
    elif train.Relevância[tweet] == 'Não relevante':
        separador = train.Treinamento[tweet].split()
        irrelevantes.extend(separador)

In [10]:
df_relevantes = pd.DataFrame(relevantes)      #DataFrame com palavras relevantes
df_relevantes.columns = ['Número de vezes que aparece em tweets relevantes']
counts_relevantes = df_relevantes['Número de vezes que aparece em tweets relevantes'].value_counts()

df_irrelevantes = pd.DataFrame(irrelevantes)  #DataFrame com palavras irrelevantes
df_irrelevantes.columns = ['Número de vezes que aparece em tweets irrelevantes']
counts_irrelevantes = df_irrelevantes['Número de vezes que aparece em tweets irrelevantes'].value_counts()

In [88]:
counts_relevantes_df = pd.DataFrame(counts_relevantes)     #DataFrame com frequências absolutas de palavras relevantes
counts_irrelevantes_df = pd.DataFrame(counts_irrelevantes) #DataFrame com frequências absolutas de palavras irrelevantes

In [158]:
 #Junção dos 2 DataFrames, para facilitar a análise. O outer faz com que junte as 2 colunas, e não misture as linhas.
counts_df =counts_relevantes_df.join(counts_irrelevantes_df, how="outer") 
counts_df.sort_values('Número de vezes que aparece em tweets relevantes', ascending=False).head()

,Número de vezes que aparece em tweets relevantes,Número de vezes que aparece em tweets irrelevantes
o,142.0,71.0
ps4,139.0,161.0
de,104.0,65.0
que,98.0,34.0
e,84.0,70.0


In [159]:
counts_df.sort_values('Número de vezes que aparece em tweets irrelevantes', ascending=False).head()

,Número de vezes que aparece em tweets relevantes,Número de vezes que aparece em tweets irrelevantes
ps4,139.0,161.0
o,142.0,71.0
e,84.0,70.0
de,104.0,65.0
um,32.0,53.0


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [166]:
#Vamos criar uma função que classifica os tweets como relevantes ou irrelevantes
def classificador_de_tweets(tweet):
    prob_relevante = 1
    prob_irrelevante = 1
    soma_relevantes = sum(counts_df['Número de vezes que aparece em tweets relevantes'])
    soma_irrelevantes = sum(counts_df['Número de vezes que aparece em tweets irrelevantes'])
    soma_total = len(counts_df)
    for palavra in cleanup(tweet):
        if palavra in counts_df.index:
            prob_relevante = prob_relevante*((counts_df['Número de vezes que aparece em tweets relevantes'][palavra])/(soma_total+soma_relevantes))
            prob_irrelevante = prob_irrelevante*((counts_df['Número de vezes que aparece em tweets irrelevantes'][palavra])/(soma_total+soma_irrelevantes))
        elif palavra not in counts_df.index:
            prob_irrelevante = prob_irrelevante*(1/(soma_total+soma_irrelevantes))
            prob_relevante = prob_relevante*(1/(soma_total+soma_irrelevantes))
    prob_relevante = prob_relevante * (relevantes_brutos_treino/tweets_totais_treino)
    prob_irrelevante = prob_irrelevante * (irrelevantes_brutos_treino/tweets_totais_treino)
           
    if prob_relevante > prob_irrelevante:
        return 'Relevante'
    elif prob_relevante < prob_irrelevante:
        return 'Não relevante'    
    else:
        return 'Erro'

In [167]:
classificador_de_tweets('o ps4 de que')

'Erro'

In [168]:
positivos_verdadeiros,positivos_falsos,negativos_verdadeiros,negativos_falsos = 0,0,0,0

for i in test.index:
    #test.Relevância = classificador_de_tweets(test.Teste[i])
    if test.Relevância[i] == 'Relevante':
        if test.Relevância[i] == classificador_de_tweets(test.Teste[i]):
            positivos_verdadeiros += 1
        elif test.Relevância[i] != classificador_de_tweets(test.Teste[i]):
            positivos_falsos += 1
    elif test.Relevância[i] == 'Não relevante':
        if test.Relevância[i] == classificador_de_tweets(test.Teste[i]):
            negativos_verdadeiros += 1
        elif test.Relevância[i] != classificador_de_tweets(test.Teste[i]):
            negativos_falsos += 1
            
print(f'O classificador identificou {positivos_verdadeiros} tweets positivos verdadeiros, com {positivos_verdadeiros/2}% dos tweets.')
print(f'O classificador identificou {positivos_falsos} tweets positivos falsos, com {positivos_falsos/2}% dos tweets.')
print(f'O classificador identificou {negativos_verdadeiros} tweets negativos verdadeiros, com {negativos_verdadeiros/2}% dos tweets.')
print(f'O classificador identificou {negativos_falsos} tweets negativos falsos, com {negativos_falsos/2}% dos tweets.')


O classificador identificou 0 tweets positivos verdadeiros, com 0.0% dos tweets.
O classificador identificou 0 tweets positivos falsos, com 0.0% dos tweets.
O classificador identificou 0 tweets negativos verdadeiros, com 0.0% dos tweets.
O classificador identificou 0 tweets negativos falsos, com 0.0% dos tweets.


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**